<a href="https://colab.research.google.com/github/xxxcrttt/Kaggle/blob/main/Titanic/Titanic_ML2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
# Titanic - Machine Learning from Disaster

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from google.colab import drive
import os
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
# Load the data
train = pd.read_csv('/content/drive/MyDrive/kaggle/titanic/train.csv')
test = pd.read_csv('/content/drive/MyDrive/kaggle/titanic/test.csv')
ID = pd.read_csv('/content/drive/MyDrive/kaggle/titanic/gender_submission.csv')

In [127]:
train = train.fillna(np.nan)
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [128]:
train['Age'] = train['Age'].fillna(train['Age'].mean())

train['Cabin'] = pd.factorize(train.Cabin)[0]

train.fillna(0, inplace=True)

train['Sex'] = [1 if x=='male' else 0 for x in train.Sex]

In [129]:
train['p1'] = np.array(train['Pclass'] == 1).astype(np.int32)
train['p2'] = np.array(train['Pclass'] == 2).astype(np.int32)
train['p3'] = np.array(train['Pclass'] == 3).astype(np.int32)

del train['Pclass']

In [130]:
train.Embarked.unique()

train['e1'] = np.array(train['Embarked'] == 1).astype(np.int32)
train['e2'] = np.array(train['Embarked'] == 2).astype(np.int32)
train['e3'] = np.array(train['Embarked'] == 3).astype(np.int32)

del train['Embarked']

In [131]:
train = train.drop(['PassengerId','Name','Ticket'], axis=1)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Sex       891 non-null    int64  
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Cabin     891 non-null    int64  
 7   p1        891 non-null    int32  
 8   p2        891 non-null    int32  
 9   p3        891 non-null    int32  
 10  e1        891 non-null    int32  
 11  e2        891 non-null    int32  
 12  e3        891 non-null    int32  
dtypes: float64(2), int32(6), int64(5)
memory usage: 69.7 KB


In [132]:
test = pd.read_csv('/content/drive/MyDrive/kaggle/titanic/test.csv')
test.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


In [133]:
test = test.fillna(np.nan)
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [134]:
test['Age'] = test['Age'].fillna(test['Age'].mean())

test['Cabin'] = pd.factorize(test.Cabin)[0]

test.fillna(0, inplace=True)

test['Sex'] = [1 if x=='male' else 0 for x in test.Sex]

In [135]:
test['p1'] = np.array(test['Pclass'] == 1).astype(np.int32)
test['p2'] = np.array(test['Pclass'] == 2).astype(np.int32)
test['p3'] = np.array(test['Pclass'] == 3).astype(np.int32)

del test['Pclass']

In [136]:
test.Embarked.unique()

test['e1'] = np.array(test['Embarked'] == 1).astype(np.int32)
test['e2'] = np.array(test['Embarked'] == 2).astype(np.int32)
test['e3'] = np.array(test['Embarked'] == 3).astype(np.int32)

del test['Embarked']

In [137]:
test = test.drop(['Name','Ticket'], axis=1)

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Sex          418 non-null    int64  
 2   Age          418 non-null    float64
 3   SibSp        418 non-null    int64  
 4   Parch        418 non-null    int64  
 5   Fare         418 non-null    float64
 6   Cabin        418 non-null    int64  
 7   p1           418 non-null    int32  
 8   p2           418 non-null    int32  
 9   p3           418 non-null    int32  
 10  e1           418 non-null    int32  
 11  e2           418 non-null    int32  
 12  e3           418 non-null    int32  
dtypes: float64(2), int32(6), int64(5)
memory usage: 32.8 KB


In [138]:
# Machine Learning Process

# split the train in to X and Y, Y = survival 
X_train = train.drop("Survived", axis=1)
Y_train = train["Survived"]
X_test  = test.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 12), (891,), (418, 12))

In [139]:
# create a function with machine learning model 
def models(X_train, Y_train):

  # Logistic Regression 
  from sklearn.linear_model import LogisticRegression
  log = LogisticRegression(random_state=0)
  log.fit(X_train, Y_train)

  # K-Nearest Neighbours
  from sklearn.neighbors import KNeighborsClassifier
  knn = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
  knn.fit(X_train, Y_train)

  # SVM(linear Kernel)
  from sklearn.svm import SVC
  svc_line = SVC(kernel='linear', random_state=0)
  svc_line.fit(X_train, Y_train)

  # SVM(RBF Kernel)
  from sklearn.svm import SVC
  svc_rbf = SVC(kernel='rbf', random_state=0)
  svc_rbf.fit(X_train, Y_train)


  # Gaissian Naive Bayes
  from sklearn.naive_bayes import GaussianNB
  nb = GaussianNB()
  nb.fit(X_train, Y_train)

  # Decision Tree
  from sklearn.tree import DecisionTreeClassifier
  tree = DecisionTreeClassifier(criterion='entropy', random_state=0)
  tree.fit(X_train, Y_train)
  y_pred = tree.predict(X_test)

  # Random Forest 
  from sklearn.ensemble import RandomForestClassifier
  forest = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
  forest.fit(X_train, Y_train)

  # print the training accuracy for each model
  print('[1]Logistic Regression Training Accuracy: ', log.score(X_train, Y_train))
  print('[2]KNN Training Accuracy: ', knn.score(X_train, Y_train))
  print('[3]SVC-linear Training Accuracy: ', svc_line.score(X_train, Y_train))
  print('[4]SVC-RBF Training Accuracy: ', svc_rbf.score(X_train, Y_train))
  print('[5]Naive Bayes Training Accuracy: ', nb.score(X_train, Y_train))
  print('[6]Decision Tree Training Accuracy: ', tree.score(X_train, Y_train))
  print('[7]Random Forest Training Accuracy: ', forest.score(X_train, Y_train))

  return log, svc_line, svc_rbf, nb, tree, forest

In [140]:
# Get and train all the models 
model = models(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[1]Logistic Regression Training Accuracy:  0.8002244668911336
[2]KNN Training Accuracy:  0.8047138047138047
[3]SVC-linear Training Accuracy:  0.7867564534231201
[4]SVC-RBF Training Accuracy:  0.6947250280583613
[5]Naive Bayes Training Accuracy:  0.7654320987654321
[6]Decision Tree Training Accuracy:  0.9865319865319865
[7]Random Forest Training Accuracy:  0.9696969696969697


In [141]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='entropy', random_state=0)
tree.fit(X_train, Y_train)
y_pred = tree.predict(X_test)

print(len(y_pred))

418


In [145]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': y_pred})

output.to_csv('submission.csv', index=False)
